In [1]:
# Day 1
# Part one
import numpy as np
data = np.loadtxt('day1.txt', dtype=int)
for j in range(2):
    data[:,j] = sorted(data[:,j])
print(np.sum(np.abs(data[:,1] - data[:,0])))

# Part two
from collections import defaultdict
numbers, counts = np.unique(data[:,1], return_counts=True)
look_up_dict = defaultdict(int)
look_up_dict.update({number:count for number, count in zip(numbers, counts)})
print(sum(d*look_up_dict[d] for d in data[:,0]))

1834060
21607792


In [2]:
# Day 2
# Part one
with open('day2.txt', 'r') as fid:
    data = [[int(d) for d in line.split()] for line in fid.read().strip().split('\n')]

import numpy as np
def is_safe(some_list):
    if len(some_list) < 1:
        return True
    current_sign = np.sign(some_list[1] - some_list[0])
    if current_sign == 0:
        return False
    for j in range(1, len(some_list)):
        diff = some_list[j] - some_list[j - 1]
        if np.sign(diff) != current_sign or abs(diff) > 3:
            return False
    return True

print(sum(is_safe(l) for l in data))

#Part two
def is_safe_when_removing_one(some_list):
    if is_safe(some_list) == True:
        return True
    for j in range(len(some_list)):
        test_list = some_list[:j] + some_list[(j + 1):]
        if is_safe(test_list) == True:
            return True
    return False
print(sum(is_safe_when_removing_one(l) for l in data))

359
418


In [3]:
# Day 3
# Part one
with open('day3.txt') as fid:
    data = fid.read().strip()
import re
pattern = r"\(\d{1,3},\d{1,3}\)"
matches = re.findall(r"mul\(\d{1,3},\d{1,3}\)", data)
s = 0
for m in matches:
    left, rght = m[4:-1].split(',')
    s += int(left)*int(rght)
print(s)

# Part two
s = 0
keywords = [r"don't()", r"do()"]
index_keyword = 0
while data:
    index = data.find(keywords[index_keyword])
    index_keyword = (index_keyword + 1) % 2
    substring, data = data[:index], data[(index + 4):]
    if index_keyword == 1:
        matches = re.findall(r"mul\(\d{1,3},\d{1,3}\)", substring)
        for m in matches:
            left, rght = m[4:-1].split(',')
            s += int(left)*int(rght)
    if index == -1:
        break
print(s)
    
    
    

196826776
106780429


In [4]:
# Day 3
# Part one
with open('day4.txt') as fid:
    lines = fid.read().strip().split('\n')


XMAS = [set(), set(), set(), set()]
for row, line in enumerate(lines):
    for col, c in enumerate(line):
        for index, c_test in enumerate('XMAS'):
            if c == c_test:
                XMAS[index].add((row, col))

def is_XMAS_in_direction_from_pos(pos, dir):
    for j in range(1, 4):
        next_pos = (pos[0] + j*dir[0], pos[1] + j*dir[1])
        if next_pos not in XMAS[j]:
            return False
    return True

all_dir = [(row, col) for row in range(-1, 2) for col in range(-1, 2)]
s = 0
for X_pos in XMAS[0]:
    for dir in all_dir:
        s += is_XMAS_in_direction_from_pos(X_pos, dir)
print(s)

# Part two
def is_MAS(A_pos, dir):
    pos1 = (A_pos[0] + dir[0], A_pos[1] + dir[1])
    pos2 = (A_pos[0] - dir[0], A_pos[1] - dir[1])
    if pos1 in XMAS[1] and pos2 in XMAS[3]:
        return True
    if pos1 in XMAS[3] and pos2 in XMAS[1]:
        return True
    return False

print(sum(is_MAS(A_pos, (1, 1)) and is_MAS(A_pos, (1, -1)) for A_pos in XMAS[2]))


2603
1965


In [5]:
# Day 5
# Part one
from collections import defaultdict
with open('day5.txt') as fid:
    rules, updates = fid.read().strip().split('\n\n')
must_not_include = defaultdict(set)
for rule in rules.split('\n'):
    left, rght = rule.split('|')
    must_not_include[int(left)].add(int(rght))

list_of_updates = [[int(pagenum) for pagenum in upd.split(',')] for upd in updates.split('\n')]

def check_update(update):
    for index, page in enumerate(update):
        for prev_page in update[:index]:
            if prev_page in must_not_include[page]:
                return False
    return True
s = 0
for update in list_of_updates:
    if check_update(update):
        s += update[len(update)//2]
print(s)

# Part two
def page_rank(update):
    ranks = [len(update) - 1]*len(update)
    for index1, page1 in enumerate(update):
        for index2, page2 in enumerate(update):
            if index1 == index2:
                continue
            if page2 in must_not_include[page1]:
                ranks[index1] -= 1
    index_middle_after_sorting = ranks.index(len(update)//2)
    return update[index_middle_after_sorting]

s = 0
for update in list_of_updates:
    if not check_update(update):
        s += page_rank(update)
print(s)

5108
7380


In [6]:
# Day 6
# Part one
obstacles = set()
with open('day6.txt','r') as fid:
    lines = fid.read().strip().split('\n')
    Ncol, Nrow = len(lines[0]), len(lines)
    for row, line in enumerate(lines):
        for col, c in enumerate(line):
            if c == '^':
                guardpos_data = (row, col)
            elif c == '#':
                obstacles.add((row, col))

directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]

dir_index = 0
guardpos = guardpos_data
visited = {guardpos}
while True:
    nextpos = (guardpos[0] + directions[dir_index][0], guardpos[1] + directions[dir_index][1])
    if not (0 <= nextpos[0] < Nrow and 0 <= nextpos[1] < Ncol):
        break
    if nextpos in obstacles:
        dir_index = (dir_index + 1) % 4
        continue
    guardpos = nextpos
    visited.add(guardpos)
print(len(visited))

# Part two
def loop_detect(new_obstacle):
    new_obstacles = {o for o in obstacles}
    new_obstacles.add(new_obstacle)
    dir_index = 0
    guardpos = guardpos_data
    exclude = {(guardpos[0], guardpos[1], dir_index)}
    while True:
        nextpos = (guardpos[0] + directions[dir_index][0], guardpos[1] + directions[dir_index][1])
        if not (0 <= nextpos[0] < Nrow and 0 <= nextpos[1] < Ncol):
            return False
        if nextpos in new_obstacles:
            dir_index = (dir_index + 1) % 4
            continue
        guardpos = nextpos
        elem = (guardpos[0], guardpos[1], dir_index)
        if elem in exclude:
            # Loop detected!
            return True
        else:
            exclude.add(elem)

s = sum(loop_detect(v) for v in visited)
print(s)

5269
1957


In [7]:
# Day 7
# Part one
from collections import deque
with open('day7.txt','r') as fid:
    lines = fid.read().strip().split('\n')
data = []
for line in lines:
    res, q = line.split(':')
    data.append((int(res), deque([int(v) for v in q.strip().split()])))
def evaluate(result, current, remaining):
    if current > result:
        return 0
    if len(remaining) == 0:
        if current == result:
            return 1
        else:
            return 0
    rhs = remaining.popleft()
    plusval = evaluate(result, current + rhs, remaining.copy())
    prodval = evaluate(result, current * rhs, remaining.copy())
    return plusval + prodval

s = 0
for result, ops in data:
    tmp = ops.copy()
    lhs = tmp.popleft()
    n = evaluate(result, lhs, tmp)
    if n > 0:
        s += result
print(s)

# Part two
def evaluate(result, current, remaining):
    if current > result:
        return 0
    if len(remaining) == 0:
        if current == result:
            return 1
        else:
            return 0
    rhs = remaining.popleft()
    plusval = evaluate(result, current + rhs, remaining.copy())
    prodval = evaluate(result, current * rhs, remaining.copy())
    concval = evaluate(result, int(str(current) + str(rhs)), remaining.copy())
    return plusval + prodval + concval

s = 0
for result, ops in data:
    tmp = ops.copy()
    lhs = tmp.popleft()
    n = evaluate(result, lhs, tmp)
    if n > 0:
        s += result
print(s)

4555081946288
227921760109726


In [8]:
# Day 8
# Part one
from collections import defaultdict
antenna_positions = defaultdict(list)
with open('day8.txt', 'r') as fid:
    lines = fid.read().strip().split('\n')
    Ncol, Nrow = len(lines[0]), len(lines)
    for row, line in enumerate(lines):
        for col, c in enumerate(line):
            if c != '.' and c != '#':
                antenna_positions[c].append((row, col))

def get_antinode_positions(ant_list):
    for index, l1 in enumerate(ant_list):
        for index2 in range(index + 1, len(ant_list)):
            l2 = ant_list[index2]
            dif = (l2[0] - l1[0], l2[1] - l1[1])
            pos1 = (l1[0] - dif[0], l1[1] - dif[1])
            pos2 = (l2[0] + dif[0], l2[1] + dif[1])
            for pos in [pos1, pos2]:
                if 0 <= pos[0] < Nrow and 0 <= pos[1] < Ncol:
                    antinode_positions.add(pos)

antinode_positions = set()
for antid, antennas in antenna_positions.items():
    get_antinode_positions(antennas)
print(len(antinode_positions))

def get_antinode_positions(ant_list):
    for index, l1 in enumerate(ant_list):
        for index2 in range(index + 1, len(ant_list)):
            l2 = ant_list[index2]
            dif = (l2[0] - l1[0], l2[1] - l1[1])
            index = 0
            while True:
                pos1 = (l1[0] - index*dif[0], l1[1] - index*dif[1])
                if 0 <= pos1[0] < Nrow and 0 <= pos1[1] < Ncol:
                    antinode_positions.add(pos1)
                else:
                    break
                index += 1
            index = 0
            while True:
                pos2 = (l2[0] + index*dif[0], l2[1] + index*dif[1])
                if 0 <= pos2[0] < Nrow and 0 <= pos2[1] < Ncol:
                    antinode_positions.add(pos2)
                else:
                    break
                index += 1

antinode_positions = set()
for antid, antennas in antenna_positions.items():
    get_antinode_positions(antennas)
print(len(antinode_positions))

400
1280


In [9]:
# Day 9
# Part one

with open('day9.txt', 'r') as fid:
    data = fid.read().strip()

backdatapointer = len(data) - 1
backdatapointer -= (backdatapointer % 2)
datapointer = 0
intdata = list(map(int, data))
N_data_sectors = sum(intdata[0::2])
s = 0
for sector in range(N_data_sectors):
    while intdata[datapointer] == 0:
        datapointer += 1
    if datapointer % 2:
        # Uneven
        j = 0
        while intdata[backdatapointer] == 0:
            backdatapointer -= 2
        fileID = backdatapointer // 2
        intdata[backdatapointer] -= 1
    else:
        # Even:
        fileID = datapointer // 2
    s += sector*fileID
    intdata[datapointer] -= 1
print(s)

# Part two
intdata = list(map(int, data))
sector_pos = 0
freespace = []
dataspace = []
for index, sector_length in enumerate(intdata):
    if index % 2:
        # Uneven, freespace:
        freespace.append((sector_pos, sector_length))
    else:
        # Even, dataspace:
        dataspace.append((sector_pos, sector_length, index // 2))
    sector_pos += sector_length

for j in range(len(dataspace) - 1, -1, -1):
    file_pos, file_length, file_ID = dataspace[j]
    for freespace_ID in range(len(freespace)):
        free_pos, free_length = freespace[freespace_ID]
        if file_pos < free_pos:
            break
        if file_length <= free_length:
            dataspace[j] = (free_pos, file_length, file_ID)
            freespace[freespace_ID] = (free_pos + file_length, free_length - file_length)
            break

s = sum(file_ID*L*(pos + (pos + L - 1))//2 for pos, L, file_ID in dataspace)
print(s)

6366665108136
6398065450842


In [10]:
# Day 10
# Part one

allpos = [set() for j in range(10)]
with open('day10.txt', 'r') as fid:
    lines = fid.read().strip().split('\n')
    Ncol, Nrow = len(lines[0]), len(lines)
    for row, line in enumerate(lines):
        for col, num in enumerate(map(int, line)):
            allpos[num].add((row, col))


def explore(current_pos, last_value):
    if current_pos not in allpos[last_value + 1]:
        return set()
    current_value = last_value + 1
    if current_value == 9:
        # Summit reached!
        return {current_pos}
    summit_pos = set()
    for dir in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
        new_pos = (current_pos[0] + dir[0], current_pos[1] + dir[1])
        summit_pos |= explore(new_pos, current_value)
    return summit_pos

s = 0
for startpos in allpos[0]:
    all_reachable_summits = explore(startpos, -1)
    s += len(all_reachable_summits)
print(s)

# Part two
def explore2(current_pos, last_value):
    if current_pos not in allpos[last_value + 1]:
        return 0
    current_value = last_value + 1
    if current_value == 9:
        # Summit reached!
        return 1
    n_trails = 0
    for dir in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
        new_pos = (current_pos[0] + dir[0], current_pos[1] + dir[1])
        n_trails += explore2(new_pos, current_value)
    return n_trails

s = 0
for startpos in allpos[0]:
    s += explore2(startpos, -1)
print(s)

811
1794


In [11]:
with open('day11.txt', 'r') as fid:
    stones = list(map(int, fid.read().strip().split()))

cache_dict = {}
def find_number_of_stones(num, level, maxlevel):
    if level == maxlevel:
        return 1
    key = (num, level, maxlevel)
    if key in cache_dict:
        return cache_dict[key]
    
    level += 1
    if num == 0:
        res = find_number_of_stones(1, level, maxlevel)
        cache_dict[key] = res
        return res
    n = len(str(num))
    if n % 2:
        res = find_number_of_stones(2024 * num, level, maxlevel)
        cache_dict[key] = res
        return res
    else:
        N = 10**(n//2)
        int1 = num//N
        int2 = num % N
        res = (find_number_of_stones(int1, level, maxlevel) + 
               find_number_of_stones(int2, level, maxlevel))
        cache_dict[key] = res
        return res

print(sum(find_number_of_stones(intj, 0, 25) for intj in stones))

# Part two
print(sum(find_number_of_stones(intj, 0, 75) for intj in stones))

199946
237994815702032
